# MongoDB Exercise

## Setup
---

Before running the following cell, make sure that you have the correct login information in the `credentials_mongodb.json` file:

> You should be careful where your credentials file is `credentials_mongodb.json`. Revise concepts on absolute path and relative path. 


In [1]:
from pymongo import MongoClient
import json
import urllib.parse

with open('credentials_mongodb.json') as f:
    login = json.load(f)

username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)
client = MongoClient(url)

## Exercise 1: Getting to know your MongoDB databases
---

List the databases that exist on your MongoDB Atlas cluster after loading sample databases. You can do this either by checking out the databases using Compass or by using `pymongo`'s `.list_database_names()` method.

In [2]:
client.list_database_names()

['sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

List the collections stored in the `sample_mflix` and `sample_airbnb` databases. You can do this either by checking out the databases using Compass or by using `pymongo`'s `.list_collection_names()` method.

In [3]:
mflix_collections = client['sample_mflix'].list_collection_names()
print('mflix DB collections: %s\n' % str(mflix_collections))
airbnb_collections = client['sample_airbnb'].list_collection_names()
print('airbnb DB collections: %s\n' % str(airbnb_collections))

mflix DB collections: ['theaters', 'movies', 'users', 'embedded_movies', 'comments', 'sessions']

airbnb DB collections: ['listingsAndReviews']



## Exercise 2: Basic MongoDB queries
---

Retrieve one (random) document associated with a movie produced in 2015.

You can use `.find_one()` method to do this, or use `.find()` but limit your results to 1 document.

In [4]:
from random import randint

movies_collection = client['sample_mflix']['movies']
# random_movie_2015 = movies_collection.find_one({'year': 2015})
count_movies_2015 = movies_collection.count_documents({'year': 2015})
random_movie_2015 = movies_collection.find(
    {'year': 2015}).skip(randint(0, count_movies_2015 - 1)).limit(1)[0]
print(random_movie_2015)

{'_id': ObjectId('573a13f7f29313caabde71b9'), 'plot': 'Human nature begins where truth ends Welcome to the murder without a body', 'genres': ['Crime', 'Drama', 'Mystery'], 'runtime': 126, 'cast': ['Elaine Jin', 'Aaron Kwok', 'Jessie Li', 'Michael Ning'], 'num_mflix_comments': 0, 'title': 'Port of Call', 'fullplot': 'Human nature begins where truth ends Welcome to the murder without a body', 'languages': ['Cantonese'], 'released': datetime.datetime(2015, 6, 26, 0, 0), 'directors': ['Philip Yung'], 'writers': ['Philip Yung'], 'awards': {'wins': 3, 'nominations': 0, 'text': '3 wins.'}, 'lastupdated': '2015-08-06 00:56:04.570000000', 'year': 2015, 'imdb': {'rating': 6.7, 'votes': 25, 'id': 4417522}, 'countries': ['Hong Kong'], 'type': 'movie'}


Retrieve all TV series produced in 1995.

> **Hint:** Inspect possible values for the `type` field.

In [5]:
# print(movies_collection.distinct('type'))   # ['movie', 'series']
tv_series = list(movies_collection.find({'year': 2015, 'type': 'series'}))
for tv in tv_series:
    print(tv)
    print()

{'_id': ObjectId('573a13dff29313caabdb76d3'), 'plot': 'The citizens of the small British town of Pagford fight for the spot on the parish council after Barry Fairbrother dies.', 'genres': ['Crime', 'Drama', 'Mystery'], 'runtime': 180, 'cast': ['Silas Carson', 'Joe Hurst', 'Michael Gambon', 'Rory Kinnear'], 'poster': 'https://m.media-amazon.com/images/M/MV5BMTkyNDgzOTcwNV5BMl5BanBnXkFtZTgwMTk4MDI1NTE@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Casual Vacancy', 'fullplot': 'When Parish Councilman Barry Fairbrother unexpectedly dies in his early forties, the town of Pagford is left in shock. An English idyll, with a cobbled market square and an ancient abbey, Pagford is not what it first seems. What lies behind the pretty facade is a town at war - rich at war with poor, teenagers at war with their parents, wives at war with their husbands, teachers at war with their pupils. And the empty seat left by Barry on the parish council soon becomes the catalyst for the biggest battle the town has y

Retrieve the title and cast of movies produced in 2010, but limit your results to 5 documents.

> **Note:** Don't forget to exclude the `_id` field from your returned documents.

> **Note:** The exact returned documents returned by `pymongo` might be different in different sessions and on different computers. This is perfectly fine.

In [6]:
movies = movies_collection.find(
    {'year': 2010},
    {'_id': 0, 'title': 1, 'cast': 1}
).limit(5)
for movie in movies:
    print(movie)
    print()

{'cast': ['èva Gèbor', 'Istvèn Znamenèk', 'èkos Horvèth', 'Lia Pokorny'], 'title': 'Pèl Adrienn'}

{'title': 'In My Sleep', 'cast': ['Philip Winchester', 'Tim Draxl', 'Lacey Chabert', 'Abigail Spencer']}

{'cast': ['James Badge Dale', 'Joseph Mazzello', 'Jon Seda', 'Sebastian Bertoli'], 'title': 'The Pacific'}

{'cast': ['Mandy Moore', 'Zachary Levi', 'Donna Murphy', 'Ron Perlman'], 'title': 'Tangled'}

{'cast': ['Nikita Mikhalkov', 'Oleg Menshikov', 'Nadezhda Mikhalkova', 'Sergey Makovetskiy'], 'title': 'Utomlyonnye solntsem 2: Predstoyanie'}



Retrieve the top 15 movies produced in 2010 that have the longest duration. Exclude TV series from your results. The returned documents should only contain the `title` and `runtime` fields (exclude the `_id` field).

> **Note:** It's ok if your results contain duplicate movies.

In [7]:
movies = movies_collection \
    .find({"year": 2010, "type": {"$ne": "series"}},
          {"_id": 0, "title": 1, "runtime": 1}) \
    .sort("runtime", -1).limit(15)

for movie in movies:
    print(movie)
    print()

{'runtime': 272, 'title': 'Mysteries of Lisbon'}

{'runtime': 181, 'title': 'Utomlyonnye solntsem 2: Predstoyanie'}

{'runtime': 181, 'title': 'Aurora'}

{'runtime': 180, 'title': 'Thorne: Sleepyhead'}

{'runtime': 180, 'title': 'The Autobiography of Nicolae Ceausescu'}

{'runtime': 178, 'title': 'Riverworld'}

{'runtime': 174, 'title': 'Enthiran'}

{'runtime': 170, 'title': 'Khaleja'}

{'runtime': 170, 'title': 'We Believed'}

{'title': 'My Name Is Khan', 'runtime': 165}

{'runtime': 163, 'title': 'Moss'}

{'runtime': 163, 'title': 'Raajneeti'}

{'runtime': 160, 'title': 'Singam'}

{'runtime': 159, 'title': 'Black Venus'}

{'runtime': 157, 'title': 'Will You Cross the Skies for Me?'}



For year 2015, return the number of movies with a metacritic rating of exactly 90.

In [8]:
count_2015_tomato_90 = movies_collection.count_documents({"year": 2015, "tomatoes.critic.meter": 90})
print(count_2015_tomato_90)

1


Retrieve the title and runtime of the 10 shortest movies in the `movies` collection.

For this exercise, you need to make sure that the field `runtime` exists in the returned documents, otherwise by default those documents would appear first which don't have a `runtime` field at all!

> **Hint:** You need the `$exists` operator (see [here](https://docs.mongodb.com/manual/reference/operator/query/exists/) for help).

In [9]:
movies = movies_collection \
    .find({"runtime": {"$exists": True}}, {"_id": 0, "title": 1, "runtime": 1}) \
    .sort("runtime", 1).limit(10)

# Print the results
for movie in movies:
    print(movie)
    print()

{'runtime': 1, 'title': 'Neko no shukai'}

{'runtime': 1, 'title': 'The Kiss'}

{'runtime': 1, 'title': 'The Kiss'}

{'runtime': 2, 'title': 'Fresh Guacamole'}

{'runtime': 2, 'title': 'Pixels'}

{'runtime': 2, 'title': 'Game Over'}

{'runtime': 2, 'title': 'Andrè and Wally B.'}

{'runtime': 2, 'title': 'Luxo Jr.'}

{'runtime': 3, 'title': 'Sisyphus'}

{'runtime': 3, 'title': 'Gagarin'}



## Exercise 3: Conditionals, embedded documents & arrays
---

Retrieve the title, production year, and number of awards of all movies that

- have been produced between 1950 and 2000 (inclusive)
- have an IMDB rating of 8.5 or better
- won at least 30 awards.

Sort the results by production year in descending order.

In [10]:
movies = movies_collection \
    .find({"year": {"$gte": 1950, "$lte": 2000}, "imdb.rating": {"$gte": 8.5}, "awards.wins": {"$gte": 30}}, 
          {"_id": 0, "title": 1, "year": 1, "awards.wins": 1}) \
    .sort("year", -1)

for movie in movies:
    print(movie)
    print()

{'year': 2000, 'title': 'Memento', 'awards': {'wins': 54}}

{'year': 2000, 'title': 'Gladiator', 'awards': {'wins': 63}}

{'year': 1999, 'title': 'The Matrix', 'awards': {'wins': 37}}

{'year': 1998, 'title': 'Saving Private Ryan', 'awards': {'wins': 83}}

{'title': 'Life Is Beautiful', 'awards': {'wins': 66}, 'year': 1997}

{'year': 1997, 'title': 'Life Is Beautiful', 'awards': {'wins': 66}}

{'year': 1995, 'title': 'The Usual Suspects', 'awards': {'wins': 36}}

{'year': 1995, 'title': 'Se7en', 'awards': {'wins': 32}}

{'year': 1994, 'title': 'Pulp Fiction', 'awards': {'wins': 64}}

{'year': 1994, 'title': 'Forrest Gump', 'awards': {'wins': 46}}

{'title': "Schindler's List", 'awards': {'wins': 81}, 'year': 1993}

{'year': 1991, 'title': 'The Silence of the Lambs', 'awards': {'wins': 56}}

{'year': 1990, 'title': 'Goodfellas', 'awards': {'wins': 43}}

{'year': 1981, 'title': 'Raiders of the Lost Ark', 'awards': {'wins': 32}}

{'year': 1977, 'title': 'Star Wars: Episode IV - A New Hope

Find the top 15 highest-rated movies according to IMDB for movies that have at least 100,000 votes. Your returned documents should only contain the `title`, `year`, and `imdb.rating` fields.

> **Hint:** Be careful about documents which have a blank space in their `imdb.rating` field!

> **Note:** It's ok if your results contain duplicates. Return 15 documents in any case.

In [11]:
movies = movies_collection \
    .find({
        "imdb.votes": {"$gte": 100000},
        "imdb.rating": {"$exists": True, "$nin": ["", " ", None, {}]}
    }, {"_id": 0, "title": 1, "year": 1, "imdb.rating": 1}) \
    .sort("imdb.rating", -1).limit(15)

for movie in movies:
    print(movie)
    print()

{'title': 'Band of Brothers', 'year': 2001, 'imdb': {'rating': 9.6}}

{'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'}

{'imdb': {'rating': 9.3}, 'year': 1994, 'title': 'The Shawshank Redemption'}

{'imdb': {'rating': 9.2}, 'year': 1972, 'title': 'The Godfather'}

{'imdb': {'rating': 9.1}, 'year': 1974, 'title': 'The Godfather: Part II'}

{'imdb': {'rating': 9.0}, 'year': 2008, 'title': 'The Dark Knight'}

{'imdb': {'rating': 8.9}, 'year': 1999, 'title': 'Fight Club'}

{'imdb': {'rating': 8.9}, 'year': 1994, 'title': 'Pulp Fiction'}

{'imdb': {'rating': 8.9}, 'year': 2003, 'title': 'The Lord of the Rings: The Return of the King'}

{'title': "Schindler's List", 'year': 1993, 'imdb': {'rating': 8.9}}

{'imdb': {'rating': 8.8}, 'year': 1980, 'title': 'Star Wars: Episode V - The Empire Strikes Back'}

{'imdb': {'rating': 8.8}, 'year': 2001, 'title': 'The Lord of the Rings: The Fellowship of the Ring'}

{'imdb': {'rating': 8.8}, 'year': 2010, 'title': 'Inception'

Retrieve the title, production year, and IMDB rating of movies in which both **Morgan Freeman** and **Clint Eastwood** played a role (among other actors in those movies). Sort the returned documents by year in descending order.

In [12]:
movies = movies_collection \
    .find({"cast": {"$all": ["Morgan Freeman", "Clint Eastwood"]}}, 
          {"_id": 0, "title": 1, "year": 1, "imdb.rating": 1}) \
    .sort("year", -1)

for movie in movies:
    print(movie)

{'imdb': {'rating': 8.1}, 'year': 2004, 'title': 'Million Dollar Baby'}
{'imdb': {'rating': 8.3}, 'year': 1992, 'title': 'Unforgiven'}


Retrieve documents associated with movies which:

- are available in both German and French (among other languages), but not in English,
- are either rated above 8 according to IMDB, or above 7.5 according to the critic ratings of [Rotten Tomatoes](https://www.rottentomatoes.com/) (inspect the `tomatoes` field),
- have at least 50 Rotten Tomatoes critic reviews.

The returned documents should include the title, year, IMDB rating, Rotten Tomatoes critic rating and country of production fields. Sort the results by IMDB rating in descending order.

**Note:** Duplicates in the results are ok.

In [13]:
filters = {
    "languages": {
        "$all": ["German", "French"],
        "$nin": ["English"]
    },
    "$or": [
        {"imdb.rating": {"$gt": 8}},
        {"tomatoes.critic.rating": {"$gt": 7.5}}
    ],
    "tomatoes.critic.numReviews": {"$gte": 50}
}

select = {
    "_id": 0,
    "title": 1,
    "year": 1,
    "imdb.rating": 1,
    "tomatoes.critic.rating": 1,
    "countries": 1
}

movies = movies_collection.find(filters, select).sort("imdb.rating", -1)

for movie in movies:
    print(movie)
    print()

{'imdb': {'rating': 7.8}, 'year': 2013, 'title': 'The Wind Rises', 'tomatoes': {'critic': {'rating': 7.9}}, 'countries': ['Japan']}

{'title': 'The Wind Rises', 'year': 2013, 'imdb': {'rating': 7.8}, 'countries': ['Japan'], 'tomatoes': {'critic': {'rating': 7.9}}}



Find the title and production year of the top 20 award-winning movies which have **not** been produced in USA, Canada, UK, or Australia.

**Note:** Duplicates are ok. Return 20 documents in any case.

In [14]:
filters = {
    "countries": {"$nin": ["USA", "Canada", "UK", "Australia"]},
    "awards.wins": {"$gte": 1}
}

select = {
    "_id": 0,
    "title": 1,
    "year": 1
}

movies = movies_collection.find(filters, select).sort("awards.wins", -1).limit(20)

for movie in movies:
    print(movie)
    print()

{'title': 'The Artist', 'year': 2011}

{'title': 'Amour', 'year': 2012}

{'title': 'Amour', 'year': 2012}

{'title': 'A Separation', 'year': 2011}

{'year': 2006, 'title': 'The Lives of Others'}

{'title': 'Let the Right One In', 'year': 2008}

{'year': 2002, 'title': 'City of God'}

{'year': 2006, 'title': 'Volver'}

{'title': 'Life Is Beautiful', 'year': 1997}

{'title': 'The Sea Inside', 'year': 2004}

{'year': 1997, 'title': 'Life Is Beautiful'}

{'year': 2001, 'title': 'Amèlie'}

{'year': 2009, 'title': 'The White Ribbon'}

{'year': 2000, 'title': 'Amores Perros'}

{'title': 'Shall We Dance?', 'year': 1996}

{'title': 'Shall We Dance?', 'year': 1996}

{'year': 2013, 'title': 'The Grandmaster'}

{'year': 1999, 'title': 'All About My Mother'}

{'year': 2001, 'title': 'Spirited Away'}

{'year': 2009, 'title': 'The Secret in Their Eyes'}

